In [1]:
import feedparser as fp
import json
import newspaper
from newspaper import Article
from time import mktime
from datetime import datetime
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
import pandas as pd
from pandas.io.json import json_normalize
import nltk
import re
from rouge import Rouge 
from collections import Counter
from geotext import GeoText
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.downloader.download('maxent_ne_chunker')
nltk.downloader.download('words')
nltk.downloader.download('treebank')
nltk.downloader.download('maxent_treebank_pos_tagger')
import string
from gensim.summarization.summarizer import summarize

from sumy.parsers.plaintext import PlaintextParser #We're choosing a plaintext parser
from sumy.nlp.tokenizers import Tokenizer 
from sumy.summarizers.lex_rank import LexRankSummarizer #We're choosing Lexrank, other algorithms are also built in
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.luhn import LuhnSummarizer

import articleDateExtractor


global str

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package maxen

In [2]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
# nltk.internals.config_java("C:/Program Files/Java/jdk1.8.0_181/bin")

import os
java_path = "C:/Program Files/Java/jdk1.8.0_181/bin/java.exe"
os.environ['JAVAHOME'] = java_path

st = StanfordNERTagger('C:/Users/anike/Downloads/stanford-ner-2018-10-16/classifiers/english.all.3class.distsim.crf.ser.gz','C:/Users/anike/Downloads/stanford-ner-2018-10-16/stanford-ner-3.9.2.jar', encoding='utf-8')


# Classifier 

In [3]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas as pd, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
import spacy
import re
import csv
from sklearn.naive_bayes import MultinomialNB
from  sklearn.metrics  import accuracy_score

C:\Users\anike\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\anike\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anike\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
lemma = nltk.WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
def preprocess(rawText):
    cleanText = re.sub(r'[^\w\s]','',rawText.lower())
    tokens = word_tokenize(cleanText)
    tokens = [token for token in tokens if not token in stop_words]
    tokens = [lemma.lemmatize(token) for token in tokens ]
    processedText = (" ".join(tokens))
    return processedText

# Importing dataset

In [6]:
data = open(r'C:\Users\anike\thailandEventCorpus_2010_2019.csv')

target, notes = [], []
reader = csv.reader(data, delimiter=",")
next(reader, None)
for i, line in enumerate(reader):
    target.append(line[1])
    rawText = line[0]
    notes.append(preprocess(rawText))

dataSet = pd.DataFrame()
dataSet['notes'] = notes
dataSet['target'] = target
dataSet = dataSet.sample(frac=1).reset_index(drop=True)

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(dataSet['notes'], dataSet['target'])

encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

vector = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
vector.fit(dataSet['notes'])
train_x =  vector.transform(train_x)
valid_x =  vector.transform(valid_x)

# Naieve Bayes Event Classifier

In [7]:

clf = MultinomialNB().fit(train_x, train_y)
predicted = clf.predict(valid_x)
print(accuracy_score(valid_y,predicted))

0.8875526157546603


# SVM Event Classifier

In [8]:
from sklearn import svm
SVM = svm.LinearSVC()
SVM.fit(train_x, train_y)
predict = SVM.predict(valid_x)
accuracy_score(valid_y,predict)

0.9476849067949489

# Random Forest Event Classifier

In [9]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=10)
rf.fit(train_x, train_y)
predict = rf.predict(valid_x)
accuracy_score(valid_y,predict)

0.9061936259771497

In [10]:
LIMIT = 4
data = {}
data['newspapers'] = {}
articlesData ={}
collections = {
    'total' : []
}


In [11]:
# with open('newspaperLinks.json') as file:
with open('newspaperLinks_Thailand.json') as file:
    sources = json.load(file)
sources

{'The Nation': {'rss': 'http://www.nationmultimedia.com/rss/politics.rss',
  'link': 'https://www.thenation.com/subject/politics/'},
 'Bangkok Post': {'rss': 'https://www.bangkokpost.com/rss/data/topstories.xml',
  'link': 'https://www.bangkokpost.com/news/politics'},
 'Khaosod English': {'rss': 'http://www.khaosodenglish.com/category/politics/feed/',
  'link': 'http://www.khaosodenglish.com/category/politics/'},
 'Intellasia': {'rss': 'https://www.intellasia.net/category/thailand/feed',
  'link': 'https://www.intellasia.net/category/thailand/feed'},
 'Thaiger': {'rss': 'https://thethaiger.com/feed',
  'link': 'https://thethaiger.com/feed'}}

In [12]:
def summarizeText(text):
    string=text
    parser = PlaintextParser.from_string(string, Tokenizer("english"))

    summarizer = LexRankSummarizer()
    summary_lex = summarizer(parser.document, 1) #Summarize the document with 5 sentences
    summary_1=''
    for line in summary_lex:
        #summary_1+=line
        #print(line)
        summary_1+=str(line)
#     print(summary_1)

#     lsa_summarizer = LsaSummarizer()
#     summary_lsa =lsa_summarizer(parser.document,3)
#     summary_2=''
#     print("\n")
#     for line in summary_lsa:
#         summary_2+=str(line)
#     print(summary_2)

#     luhn_summarizer = LuhnSummarizer() 
#     luhn_summarizer.stop_words = ("I",  "me", "is", "am", "the","than", "that", "this","an","you", "are")
#     summary_3=''
#     print("\n")
#     for line in luhn_summarizer(parser.document, 3):
#         summary_3+=str(line)
#     print(summary_3)
    return summary_1

def getLocation(ents, text):
    loc = ""
#     locList = []
    items = [x.text for x in ents if x.label_ in 'GPE' or x.label_ in 'LOC']
    loc_ = [Counter(items).most_common(1)]
    places = GeoText(text)
    topPlace = [Counter(places.cities).most_common(1)]
    
    places = (places.cities)
#     text = re.sub(r'[^\w\s]','',text)
#     tokenized_text = word_tokenize(text)
#     classified_text = st.tag(tokenized_text)
#     stanfordLocations =[]
#     for x in classified_text:
#         if(x[1]=="LOCATION"):
#             stanfordLocations.append(x[0])
#     topPlace = [Counter(stanfordLocations).most_common(1)]
#     locList = [i for i in stanfordLocations if i in items]
    locList = [i for i in places if i in items]
    total = items+locList 
    location_ = [Counter(total).most_common(1)]
    if len(location_) == 0:
        if(len(loc_))!=0:
            loc = loc_[0][0][0]
        elif (len(topPlace)!=0):
            loc = topPlace[0]
        else:
            loc = ""
    else:
        if(len(location_))!=0:
            if(len(location_[0]))!=0:
                if(len(location_[0][0]))!=0:
                    loc = location_[0][0][0]
        else:
            loc = "unknown"
    return loc

def getEventType(tokens):
#     eventTypes ={
#     'civilianViolence':['rebel','violence','militia','war','rape','torture','kill','shoot','fight','murder','molest','attack','terror','battle','assault','blood'],
#     'riotsProtest':['protest','riot']
#     }
    eventType = "other"
    for event, types in eventTypes.items():
        tag = [token for token in tokens if any(typ in token for typ in types)]
        if len(tag) != 0:
            eventType = event
            break
    return eventType

# News Scraper and Classifier

In [20]:
collections = {
    'total' : []
}
from time import mktime
from datetime import datetime
from lexrank import STOPWORDS, LexRank

event_type = ""
data = {}
data['newspapers'] = {}
articlesData ={}
collectedNews = {}
events = ['arrest','arrested','violence against civilians','rioting','protesting','riots','protest','battles','violence','remote violence','assault','attack','bloodshed','brutality','clash','confusion','cruelty','disorder','disturbance','fighting','rampage','struggle','terrorism','abandon','acuteness','bestiality','blowup','coercion','compulsion','constraint','destructiveness','duress','ferocity','fervor','fierceness','flap','frenzy','fury','fuss','harshness','murderousness','onslaught','passion','power','roughness','ruckus','rumble','savagery','scam','severity','sharpness','storm','storminess','tumult','turbulence','uproar','vehemence','wildness','brute','force','foul','play','raging','anarchy','brawl','disturbance','lawlessness','protest','storm','strife','trouble','turbulence','turmoil','uproar','anarchism','burst','commotion','confusion','distemper','flap','fray','free-for-all','fuss','hassle','misrule','mix-up','quarrel','racket','row','ruckus','ruction','rumble','rumpus','run-in','scene','shivaree','shower','snarl','stir','to-do','tumult','brannigan','mob','violence','street','fighting','wingding','action','assault','attack','bloodshed','bombing','campaign','clash','combat','conflict','crusade','encounter','fighting','hostility','skirmish','strife','struggle','war','warfare','barrage','brush','carnage','contention','engagement','fray','havoc','onset','onslaught','press','ravage','scrimmage','sortie','blitzkreig']
politics= ["politicize", "political affairs", "administration", "thailand politics", "politicians", "elections", "votes", "voters", 'malay muslim separatist', 'pheu thai', 'palang pracharath', 'future forward', 'democrat', 'thai liberal', 'bhumjaithai', 'chartthaipattana']
violence = ["violence", "criminal violence", "criminal", "crime", "criminal party", "election violence", "electoral violence","mob killing", "vigilante", "riots", "killing" ]
parties = ['malay muslim separatist', 'pheu thai', 'pheu','palang pracharath', 'future forward', 'democrat', 'thai liberal', 'bhumjaithai', 'chartthaipattana']
        
eventTypes ={
'riots':['riot'],
'protests':['protest','march'], 
'civilianViolence':['rebel','violence','militia','war','rape','torture','kill','shoot','fight','murder','molest','attack','terror','battle','assault','blood']
}



count = 1

for source, link in sources.items():
    print(source)
    count = 1
    if 'rss' in link:
        parsedFeed = fp.parse(link['rss'])
        collected = {
            'rss': link['rss'],
            'link': link['link'],
            'article': []
        }
        for feed in parsedFeed.entries:
            
            try:

                articlesData ={}
                articles = Article(feed['links'][0]['href'])
                articles.download()
                articles.parse()
                articles.nlp()
                strg = re.sub(r'[^\w\s]','',articles.text)
                articleTokens = nlp(strg)

                tokens =[]
                for token in articleTokens:
                    tokens.append(token.text.lower())

                txt = str(articles.text).split()
                keyWords = articles.keywords

                eventType = "other"
                if(([a for a in keyWords if any(b.lower() in a.lower() for b in politics)])):
                    print(articles.title)
                    articlesData['title'] = articles.title

                    article = nlp(articles.text)
                    ###Summary
                    sentences = [x for x in article.sents]
                    sentences = list( map(str, sentences) )
                    lxr = LexRank(articles.text)
                    lxr.get_summary(sentences)
                    summary = lxr.get_summary(sentences, summary_size=1, threshold=.1)
                    summary = (" ".join(summary))
                    
                    rawSummary = [preprocess(summary)]
                    textVector = vector.transform(rawSummary)
                    prediction = rf.predict(textVector)
                    print(encoder.inverse_transform(prediction))
                    articlesData['event'] = encoder.inverse_transform(prediction[0])

                    dates = articleDateExtractor.extractArticlePublishedDate(feed['links'][0]['href'])
                    localFormat = dates.strftime("%m/%d/%Y")
                    summaryDate = dates.strftime("On %d %b,")
                    articlesData['eventDate'] = localFormat
                    articlesData['location'] = getLocation(articleTokens.ents, articles.text)
                    orgs = set(keyWords).intersection(set(parties))

                    if len(orgs) == 0:
                        if(articlesData['event']=="Riots"):
                            taggedOrg = "Rioters"
                        elif(articlesData['event']=="Protests"):
                            taggedOrg = "Protestors"
                        else:
                            taggedOrg = "other"
                    else:
                        taggedOrg=""
                        for org in orgs:
                            taggedOrg+=org+", "
                        taggedOrg=taggedOrg[:-2]

                    articlesData['partiesInvolved'] = taggedOrg
                    articlesData['source'] = source
                    totalSummary = []
                    totalSummary.append(summaryDate)
                    totalSummary.append(summary)
                    extractiveSummary  = (" ".join(totalSummary))
                    articlesData['summary'] = extractiveSummary
                    articlesData['link'] = (feed['links'][0]['href'])
                    collected['article'].append(articlesData)
#                 else:
#                     break
            except Exception as ex:
                print(ex)
        collections['total'].append(collected)
    else:
        parsedFeed = newspaper.build(link['link'], memoize_articles = False)
        collected = {
            'link': link['link'],
            'article': []
        }
        for feed in parsedFeed.articles:
#             if count > 1000:
#                 break
            try:
                feed.download()
                feed.parse()
                feed.nlp()
                if feed.publish_date is not None:
                    articlesData ={}
                    
                    strg = re.sub(r'[^\w\s]','',feed.text)
                    articleTokens = nlp(strg)
                    tokens =[]
                    for token in articleTokens:
                        tokens.append(token.text.lower())

                        txt = str(feed.text).split()
                        for i in txt:
                            for j in events:
                                if i==j:
                                    event_type = j
                                    break
                                else:
                                    event_type = "other"
                    keyWords = feed.keywords
                    if(([a for a in keyWords if any(b.lower() in a.lower() for b in politics)])):
                        
                        articlesData['title'] = feed.title
                        article = nlp(feed.text)
                        ###Summary
                        sentences = [x for x in article.sents]
                        sentences = list( map(str, sentences) )
                        lxr = LexRank(feed.text)
                        lxr.get_summary(sentences)
                        summary = lxr.get_summary(sentences, summary_size=1, threshold=.1)
                        summary = (" ".join(summary))

                        rawSummary = [preprocess(summary)]
                        textVector = vector.transform(rawSummary)
                        prediction = rf.predict(textVector)
                        print(encoder.inverse_transform(prediction))
                        articlesData['event'] = encoder.inverse_transform(prediction[0])
                        
#                         articlesData['event'] = getEventType(tokens)
                        articlesData['eventDate'] = feed.publish_date.strftime('%m/%d/%Y')
                        articlesData['location'] = getLocation(articleTokens.ents,feed.text)
                        
                        orgs = set(keyWords).intersection(set(parties))
                        if len(orgs) == 0:
                            if(articlesData['event']=="Riots"):
                                taggedOrg = "Rioters"
                            elif(articlesData['event']=="Protests"):
                                taggedOrg = "Protestors"
                            else:
                                taggedOrg = "other"
                        else:
                            taggedOrg=""
                            for org in orgs:
                                taggedOrg+=org+", "
                            taggedOrg=taggedOrg[:-2]

                        articlesData['partiesInvolved'] = taggedOrg
                        articlesData['source'] = source
                        totalSummary = []
                        totalSummary.append(feed.publish_date.strftime("On %d %b,"))
                        totalSummary.append(summary)
                        extractiveSummary  = (" ".join(totalSummary))
                        articlesData['summary'] = extractiveSummary
                        articlesData['link'] = link
                        collected['article'].append(articlesData)
#                     else:
#                         break
#                         count +=1
            except Exception as ex:
                print(ex)
        collections['total'].append(collected)

The Nation
EC party-list calculation opens a can of worms


C:\Users\anike\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


['Violence against civilians']


C:\Users\anike\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Extracting date from http://www.nationmultimedia.com/detail/politics/30369064
EC endorses 349 candidates but warns not all candidates in the clear


...

Extracting date from https://thethaiger.com/hot-news/elections/democrats-poised-to-take-on-the-election-commission-over-party-list-seats
Exception in extractArticlePublishedDate for https://thethaiger.com/hot-news/elections/democrats-poised-to-take-on-the-election-commission-over-party-list-seats
()
'NoneType' object has no attribute 'strftime'
Forming the next Thai government - Bhumjaithai and Democrats will decide


C:\Users\anike\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


['Protests']


C:\Users\anike\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Extracting date from https://thethaiger.com/hot-news/elections/forming-the-next-thai-government-bhumjaithai-and-democrats-will-decide
Exception in extractArticlePublishedDate for https://thethaiger.com/hot-news/elections/forming-the-next-thai-government-bhumjaithai-and-democrats-will-decide
()
'NoneType' object has no attribute 'strftime'


In [21]:
collections["total"]

[{'rss': 'http://www.nationmultimedia.com/rss/politics.rss',
  'link': 'https://www.thenation.com/subject/politics/',
  'article': [{'title': 'EC party-list calculation opens a can of worms',
    'event': 'Violence against civilians',
    'eventDate': '05/08/2019',
    'location': 'Chiang Mai',
    'partiesInvolved': 'other',
    'source': 'The Nation',
    'summary': 'On 08 May, The Election Commission (EC) announced yesterday that 26 parties had secured party-list MP seats despite 11 of the parties not winning enough votes to be entitled to the seats and only 16 being eligible.\n\n',
    'link': 'http://www.nationmultimedia.com/detail/politics/30369064'},
   {'title': 'EC endorses 349 candidates but warns not all candidates in the clear',
    'event': 'Other',
    'eventDate': '05/07/2019',
    'location': 'Chiang Mai province',
    'partiesInvolved': 'other',
    'source': 'The Nation',
    'summary': 'On 07 May, The use of this method would come at the expense of Future Forward, wh

In [22]:
frame = json_normalize(collections['total'])
rows = frame.shape[0]
frame

,article,link,rss
0,[{'title': 'EC party-list calculation opens a ...,https://www.thenation.com/subject/politics/,http://www.nationmultimedia.com/rss/politics.rss
1,[],https://www.bangkokpost.com/news/politics,https://www.bangkokpost.com/rss/data/topstorie...
2,[{'title': 'EC Admits 27 Parties to Parliament...,http://www.khaosodenglish.com/category/politics/,http://www.khaosodenglish.com/category/politic...
3,[],https://www.intellasia.net/category/thailand/feed,https://www.intellasia.net/category/thailand/feed
4,[],https://thethaiger.com/feed,https://thethaiger.com/feed


In [23]:
totalDF = pd.DataFrame()
for row in range(0,rows):
    tempDF = pd.DataFrame()
    tempDF = json_normalize(collections['total'][row]['article'])
    totalDF = totalDF.append(tempDF, ignore_index = True) 


# Dataframe that contains scraped news which has been classified and summarized

In [24]:
totalDF

,event,eventDate,link,location,partiesInvolved,source,summary,title
0,Violence against civilians,05/08/2019,http://www.nationmultimedia.com/detail/politic...,Chiang Mai,other,The Nation,"On 08 May, The Election Commission (EC) announ...",EC party-list calculation opens a can of worms
1,Other,05/07/2019,http://www.nationmultimedia.com/detail/politic...,Chiang Mai province,other,The Nation,"On 07 May, The use of this method would come a...",EC endorses 349 candidates but warns not all c...
2,Other,05/02/2019,http://www.nationmultimedia.com/detail/politic...,Palang Rak,other,The Nation,"On 02 May, Their votes would not be used in th...",11 election candidates disqualified
3,Protests,05/08/2019,http://www.khaosodenglish.com/politics/2019/05...,BANGKOK,Protestors,Khaosod English,"On 08 May, Since 35,532,647 voters participate...","EC Admits 27 Parties to Parliament, Setting Ne..."
4,Protests,05/08/2019,http://www.khaosodenglish.com/politics/2019/05...,BANGKOK,Protestors,Khaosod English,"On 08 May, Future Foward, which placed third i...",Court Affirms EC’s Power to Allocate Party-Lis...
5,Other,04/30/2019,http://www.khaosodenglish.com/politics/2019/04...,BANGKOK,other,Khaosod English,"On 30 Apr, But under the latest formula being ...",Future Forward May Sue EC For Dereliction of D...
6,Protests,04/29/2019,http://www.khaosodenglish.com/politics/2019/04...,,democrat,Khaosod English,"On 29 Apr, The controversy is the latest to do...",Future Forward Calls for Re-Vote to Settle Con...
7,Protests,04/29/2019,http://www.khaosodenglish.com/politics/2019/04...,Srisuwan,Protestors,Khaosod English,"On 29 Apr, Power was restored about 30 seconds...",Democrats Dispute Future Forward Victory After...
8,Other,04/24/2019,http://www.khaosodenglish.com/politics/2019/04...,BANGKOK,pheu,Khaosod English,"On 24 Apr, He said he wasn’t involved in organ...",By-Election Ordered in Chiang Mai as Pheu Thai...
9,Protests,04/23/2019,http://www.khaosodenglish.com/politics/2019/04...,BANGKOK,Protestors,Khaosod English,"On 23 Apr, Earlier on Tuesday, Future Forward ...",Election Commission Accuses Thanathorn of Brea...


In [25]:
import datetime
now = datetime.datetime.now()
totalDF.to_excel("airData/thailand/testing_"+now.strftime("%m%d%Y_%H%M%S")+".xlsx")